In [1]:
from u2net import U2NET
import torch
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
from matplotlib import pyplot as plt

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = U2NET(3, 1)  # 3 input channels, 1 output channel
model_path = 'saved_models/u2net/u2net.pth'

In [7]:
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval() 

U2NET(
  (stage1): RSU7(
    (rebnconvin): REBNCONV(
      (conv_s1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (rebnconv1): REBNCONV(
      (conv_s1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv2): REBNCONV(
      (conv_s1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv3): REBNCONV(
      (conv_s1): Conv2d(32, 32, k

In [8]:
def preprocess_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((320, 320)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    input_tensor = transform(image).unsqueeze(0)
    return input_tensor, image.size

def postprocess_mask(mask, original_size):
    mask = mask.squeeze().cpu().detach().numpy()
    mask = cv2.resize(mask, original_size, interpolation=cv2.INTER_LINEAR)
    mask = (mask * 255).astype(np.uint8)
    return mask

def generate_alpha_mask(image_path):
    input_tensor, original_size = preprocess_image(image_path)
    
    with torch.no_grad():
        output = model(input_tensor)[0]
    
    alpha_mask = postprocess_mask(output, original_size)
    return alpha_mask

In [9]:
image_path = 'Screenshot (776).png'  # Replace with the path to your image
alpha_mask = generate_alpha_mask(image_path)

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [10]:
plt.imshow(alpha_mask)
plt.show()

NameError: name 'alpha_mask' is not defined

In [7]:
cv2.imwrite("alpha_mask_output10.png", alpha_mask)

True